In [2]:
# Import all required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("✓ All libraries imported successfully!")
print("✓ Ready for data preprocessing")


✓ All libraries imported successfully!
✓ Ready for data preprocessing


In [3]:
# Load your master CSV file
# Replace 'your_file_path.csv' with the actual path to your dataset
file_path = 'combined_cutoffs.csv'  # UPDATE THIS PATH

try:
    df = pd.read_csv(file_path)
    print(f"✓ Dataset loaded successfully!")
    print(f"Dataset shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
    
    # Display first few rows
    print("\nFirst 5 rows:")
    display(df.head())
    
except FileNotFoundError:
    print("❌ File not found. Please update the file_path variable with correct path.")
    print("Expected columns: College_Code, College_Name, Category, Branch, Cutoff_Rank, Year, Round, Exam_Type")
except Exception as e:
    print(f"❌ Error loading dataset: {str(e)}")
    print("Please check your file path and file format.")


✓ Dataset loaded successfully!
Dataset shape: (309145, 8)
Columns: ['College_Code', 'College_Name', 'Category', 'Branch', 'Cutoff_Rank', 'Year', 'Round', 'Exam_Type']

First 5 rows:


,College_Code,College_Name,Category,Branch,Cutoff_Rank,Year,Round,Exam_Type
0,E001,Acharya Institute of Technology,GM,AE-Aeronautical Engineering,63649.0,2023,4,COMEDK
1,E001,Acharya Institute of Technology,GM,AI-Artificial Intelligence & Machine Learning,27873.0,2023,4,COMEDK
2,E001,Acharya Institute of Technology,GM,BT-Biotechnology,46368.0,2023,4,COMEDK
3,E001,Acharya Institute of Technology,GM,CD-Computer Science & Engineering (Data Science),24270.0,2023,4,COMEDK
4,E001,Acharya Institute of Technology,GM,CS-Computer Science & Engineering,20937.0,2023,4,COMEDK


In [4]:
# Comprehensive data exploration
print("=== DATASET OVERVIEW ===")
print(f"Shape: {df.shape}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024:.2f} KB")

print("\n=== DATA TYPES ===")
print(df.dtypes)

print("\n=== MISSING VALUES ANALYSIS ===")
missing_data = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df)) * 100,
    'Data_Type': df.dtypes
})

missing_data = missing_data[missing_data['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)
print("Missing values summary:")
print(missing_data)

if len(missing_data) == 0:
    print("✓ No missing values found!")

print("\n=== UNIQUE VALUES PER COLUMN ===")
for col in df.columns:
    unique_count = df[col].nunique()
    print(f"{col}: {unique_count} unique values")
    if unique_count <= 20 and col in ['Category', 'Round', 'Exam_Type']:
        print(f"  Values: {sorted(df[col].dropna().unique())}")
    elif col == 'Branch' and unique_count <= 50:
        branch_values = sorted(df[col].dropna().unique())
        print(f"  Sample branches: {branch_values[:10]}...")
        if len(branch_values) > 10:
            print(f"  Total branches: {len(branch_values)}")

print("\n=== BASIC STATISTICS FOR NUMERICAL COLUMNS ===")
print(df.describe())


=== DATASET OVERVIEW ===
Shape: (309145, 8)
Memory usage: 115870.97 KB

=== DATA TYPES ===
College_Code     object
College_Name     object
Category         object
Branch           object
Cutoff_Rank     float64
Year              int64
Round             int64
Exam_Type        object
dtype: object

=== MISSING VALUES ANALYSIS ===
Missing values summary:
                  Column  Missing_Count  Missing_Percentage Data_Type
Category        Category           3287            1.063255    object
Branch            Branch           3067            0.992091    object
Cutoff_Rank  Cutoff_Rank           3067            0.992091   float64

=== UNIQUE VALUES PER COLUMN ===
College_Code: 347 unique values
College_Name: 822 unique values
Category: 65 unique values
Branch: 490 unique values
Cutoff_Rank: 118830 unique values
Year: 5 unique values
Round: 5 unique values
  Values: [0, 1, 2, 3, 4]
Exam_Type: 2 unique values
  Values: ['CET', 'COMEDK']

=== BASIC STATISTICS FOR NUMERICAL COLUMNS ===
       

In [5]:
def handle_missing_values_admission_data(df):
    """
    Handle missing values specifically for college admission dataset
    Adapted for your specific data structure
    """
    df_cleaned = df.copy()
    
    print("=== MISSING VALUES HANDLING STRATEGY ===\n")
    
    # Check which columns actually have missing values
    missing_cols = df_cleaned.isnull().sum()
    missing_cols = missing_cols[missing_cols > 0]
    
    if len(missing_cols) == 0:
        print("✓ No missing values found!")
        return df_cleaned
    
    print(f"Columns with missing values: {list(missing_cols.index)}")
    
    # 1. Handle Category missing values
    if 'Category' in missing_cols.index:
        print("\n1. Handling Category:")
        category_missing = df_cleaned['Category'].isnull()
        
        # Check most common category in your dataset
        most_common_category = df_cleaned['Category'].mode().iloc[0] if len(df_cleaned['Category'].mode()) > 0 else 'GM'
        
        # Fill missing categories with most common (likely 'GM' for General Merit)
        df_cleaned['Category'] = df_cleaned['Category'].fillna(most_common_category)
        print(f"   Filled {category_missing.sum()} missing Categories with '{most_common_category}'")
        
        # Display category distribution
        print("   Top 10 categories in dataset:")
        print(df_cleaned['Category'].value_counts().head(10))
    
    # 2. Handle Branch missing values
    if 'Branch' in missing_cols.index:
        print("\n2. Handling Branch:")
        branch_missing_before = df_cleaned['Branch'].isnull().sum()
        
        # For missing branches, use most common branch from same college
        for idx in df_cleaned[df_cleaned['Branch'].isnull()].index:
            college_code = df_cleaned.loc[idx, 'College_Code']
            same_college_branches = df_cleaned[
                (df_cleaned['College_Code'] == college_code) & 
                (df_cleaned['Branch'].notna())
            ]['Branch']
            
            if len(same_college_branches) > 0:
                # Use most common branch in same college
                common_branch = same_college_branches.mode().iloc[0]
                df_cleaned.loc[idx, 'Branch'] = common_branch
            else:
                # Use overall most common branch
                overall_common = df_cleaned['Branch'].mode().iloc[0] if len(df_cleaned['Branch'].mode()) > 0 else 'CS'
                df_cleaned.loc[idx, 'Branch'] = overall_common
        
        branch_missing_after = df_cleaned['Branch'].isnull().sum()
        print(f"   Filled {branch_missing_before - branch_missing_after} missing Branches")
        
        # Show top branches
        print("   Top 10 branches in dataset:")
        print(df_cleaned['Branch'].value_counts().head(10))
    
    # 3. Handle Cutoff_Rank missing values
    if 'Cutoff_Rank' in missing_cols.index:
        print("\n3. Handling Cutoff_Rank:")
        rank_missing_before = df_cleaned['Cutoff_Rank'].isnull().sum()
        
        # Strategy: Use median rank for similar records
        for idx in df_cleaned[df_cleaned['Cutoff_Rank'].isnull()].index:
            branch = df_cleaned.loc[idx, 'Branch']
            category = df_cleaned.loc[idx, 'Category']
            exam_type = df_cleaned.loc[idx, 'Exam_Type']
            year = df_cleaned.loc[idx, 'Year']
            
            # Find similar records (same branch, category, exam type, year)
            similar_records = df_cleaned[
                (df_cleaned['Branch'] == branch) & 
                (df_cleaned['Category'] == category) & 
                (df_cleaned['Exam_Type'] == exam_type) & 
                (df_cleaned['Year'] == year) &
                (df_cleaned['Cutoff_Rank'].notna())
            ]['Cutoff_Rank']
            
            if len(similar_records) > 0:
                fill_value = similar_records.median()
                df_cleaned.loc[idx, 'Cutoff_Rank'] = fill_value
            else:
                # Fallback: Use branch median for same year and exam type
                branch_similar = df_cleaned[
                    (df_cleaned['Branch'] == branch) & 
                    (df_cleaned['Year'] == year) &
                    (df_cleaned['Exam_Type'] == exam_type) &
                    (df_cleaned['Cutoff_Rank'].notna())
                ]['Cutoff_Rank']
                
                if len(branch_similar) > 0:
                    df_cleaned.loc[idx, 'Cutoff_Rank'] = branch_similar.median()
                else:
                    # Final fallback: Overall median
                    overall_median = df_cleaned['Cutoff_Rank'].median()
                    df_cleaned.loc[idx, 'Cutoff_Rank'] = overall_median
        
        rank_missing_after = df_cleaned['Cutoff_Rank'].isnull().sum()
        print(f"   Filled {rank_missing_before - rank_missing_after} missing Cutoff_Ranks")
    
    # 4. Handle any other missing values
    remaining_missing = df_cleaned.isnull().sum()
    remaining_missing = remaining_missing[remaining_missing > 0]
    
    if len(remaining_missing) > 0:
        print(f"\n4. Remaining missing values:")
        print(remaining_missing)
    
    return df_cleaned

print("✓ Missing values handling function defined")


✓ Missing values handling function defined


In [6]:
# Apply missing values handling to your dataset
print("APPLYING MISSING VALUES HANDLING:")
print("=" * 50)

# Store original data info for comparison
original_shape = df.shape
original_missing = df.isnull().sum().sum()

print(f"Original dataset:")
print(f"  Shape: {original_shape}")
print(f"  Total missing values: {original_missing}")

# Apply the function
df_cleaned = handle_missing_values_admission_data(df)

# Summary of changes
cleaned_missing = df_cleaned.isnull().sum().sum()

print(f"\n=== MISSING VALUES HANDLING SUMMARY ===")
print(f"Original missing values: {original_missing}")
print(f"Remaining missing values: {cleaned_missing}")
print(f"Missing values resolved: {original_missing - cleaned_missing}")

print("\nMissing values by column after cleaning:")
missing_summary = df_cleaned.isnull().sum()
print(missing_summary)

if cleaned_missing == 0:
    print(f"\n✅ All missing values handled successfully!")
else:
    print(f"\n⚠️ {cleaned_missing} missing values remain")

print(f"\n✅ Dataset ready for feature engineering")
print(f"Cleaned dataset shape: {df_cleaned.shape}")


APPLYING MISSING VALUES HANDLING:
Original dataset:
  Shape: (309145, 8)
  Total missing values: 9421
=== MISSING VALUES HANDLING STRATEGY ===

Columns with missing values: ['Category', 'Branch', 'Cutoff_Rank']

1. Handling Category:
   Filled 3287 missing Categories with 'GM'
   Top 10 categories in dataset:
Category
GM     35884
2AG    19795
SCG    19657
GMR    19304
3BG    17842
GMH    17087
3AG    16995
2BG    16943
1G     16362
STG    15188
Name: count, dtype: int64

2. Handling Branch:
   Filled 3067 missing Branches
   Top 10 branches in dataset:
Branch
CS Computers                    70033
EC Electronics                  51208
IE Info.Science                 27884
EE Electrical                   19907
CE Civil                        18614
AI Artificial Intelligence      17412
ME Mechanical                   13602
CA CS (AI, Machine Learning)     8189
DS Comp. Sc. Engg- Data Sc.      7638
AD Artificial Intel, Data Sc     6626
Name: count, dtype: int64

3. Handling Cutoff_Rank:
 

In [7]:
# Feature Engineering - Create additional useful features
print("=== FEATURE ENGINEERING ===")

df_features = df_cleaned.copy()

# 1. Rank Percentile - Normalized rank within year/exam type
print("1. Creating Rank_Percentile feature:")
df_features['Rank_Percentile'] = df_features.groupby(['Year', 'Exam_Type'])['Cutoff_Rank'].rank(pct=True)
print("   ✓ Rank_Percentile: Normalized rank within year and exam type (0-1 scale)")

# 2. Years since 2020 - Temporal encoding
print("\n2. Creating Years_Since_2020 feature:")
df_features['Years_Since_2020'] = df_features['Year'] - 2020
print("   ✓ Years_Since_2020: Temporal trend encoding")

# 3. Branch popularity score (based on cutoff rank - lower rank = more popular)
print("\n3. Creating Branch_Popularity_Score:")
branch_popularity = df_features.groupby('Branch')['Cutoff_Rank'].median().sort_values()
branch_popularity_rank = {branch: rank for rank, branch in enumerate(branch_popularity.index)}
df_features['Branch_Popularity_Score'] = df_features['Branch'].map(branch_popularity_rank)
print(f"   ✓ Branch_Popularity_Score: Ranked {len(branch_popularity_rank)} branches by median cutoff rank")
print(f"   Top 5 most popular branches (lowest cutoff): {list(branch_popularity.head().index)}")

# 4. College selectivity (average cutoff rank per college - lower = more selective)
print("\n4. Creating College_Selectivity:")
college_selectivity = df_features.groupby('College_Code')['Cutoff_Rank'].median()
df_features['College_Selectivity'] = df_features['College_Code'].map(college_selectivity)
print(f"   ✓ College_Selectivity: Median cutoff rank per college")

# 5. Round difficulty (based on your explanation)
print("\n5. Creating Round_Difficulty:")
# Higher rounds typically have higher (worse) cutoff ranks
round_difficulty_map = {0: 0, 1: 1, 2: 2, 3: 3, 4: 4}  # Direct mapping for now
df_features['Round_Difficulty'] = df_features['Round'].map(round_difficulty_map)
print("   ✓ Round_Difficulty: Round progression indicator")

# 6. Category type simplification (extract main category)
print("\n6. Creating Category_Main:")
def extract_main_category(cat):
    if 'GM' in str(cat):
        return 'General'
    elif any(x in str(cat) for x in ['SC', '2A', '2B', '3A', '3B']):
        return 'Reserved'
    elif 'ST' in str(cat):
        return 'ST'
    else:
        return 'Other'

df_features['Category_Main'] = df_features['Category'].apply(extract_main_category)
print("   ✓ Category_Main: Simplified category grouping")
print(f"   Category distribution: {df_features['Category_Main'].value_counts().to_dict()}")

print(f"\n=== FEATURE ENGINEERING SUMMARY ===")
print(f"Original features: {df_cleaned.shape[1]}")
print(f"After feature engineering: {df_features.shape[1]}")
print(f"New features added: {df_features.shape[1] - df_cleaned.shape[1]}")
new_features = [col for col in df_features.columns if col not in df_cleaned.columns]
print(f"New feature names: {new_features}")

# Display sample of new features
print("\nSample of engineered features (first 5 rows):")
sample_cols = ['College_Code', 'Branch', 'Cutoff_Rank', 'Rank_Percentile', 'Years_Since_2020', 
               'Branch_Popularity_Score', 'College_Selectivity', 'Round_Difficulty', 'Category_Main']
print(df_features[sample_cols].head())

print(f"\n✅ Feature engineering completed successfully!")


=== FEATURE ENGINEERING ===
1. Creating Rank_Percentile feature:
   ✓ Rank_Percentile: Normalized rank within year and exam type (0-1 scale)

2. Creating Years_Since_2020 feature:
   ✓ Years_Since_2020: Temporal trend encoding

3. Creating Branch_Popularity_Score:
   ✓ Branch_Popularity_Score: Ranked 490 branches by median cutoff rank
   Top 5 most popular branches (lowest cutoff): ['Computer\rScience and\rEngineering\r(Block Chain)', 'Computer Science\rand Engineering\r(Cyber Security)', 'Electronics &\rInstrumentat\rion\rEngineering', 'Computer Science\rand Business\rSystems', 'Computer Science\rand Business Systems']

4. Creating College_Selectivity:
   ✓ College_Selectivity: Median cutoff rank per college

5. Creating Round_Difficulty:
   ✓ Round_Difficulty: Round progression indicator

6. Creating Category_Main:
   ✓ Category_Main: Simplified category grouping
   Category distribution: {'Reserved': 167204, 'General': 93208, 'Other': 25346, 'ST': 23387}

=== FEATURE ENGINEERING SUM

In [8]:
def encode_categorical_features(df, encoding_strategy='comprehensive'):
    """
    Apply appropriate categorical encoding for college admission prediction
    Adapted for your specific dataset with 490 branches and detailed categories
    """
    df_encoded = df.copy()
    encoders = {}
    
    print("=== CATEGORICAL ENCODING STRATEGY ===\n")
    
    # 1. College_Code - Label encoding (347 unique colleges)
    print("1. College_Code Encoding:")
    le_college_code = LabelEncoder()
    df_encoded['College_Code_Encoded'] = le_college_code.fit_transform(df_encoded['College_Code'])
    df_encoded = df_encoded.drop('College_Code', axis=1)
    encoders['college_code'] = le_college_code
    print(f"   Label encoded College_Code -> College_Code_Encoded")
    print(f"   Unique colleges: {len(le_college_code.classes_)}")
    
    # 2. College_Name - Drop (redundant with College_Code)
    print("\n2. College_Name Encoding:")
    df_encoded = df_encoded.drop('College_Name', axis=1)
    print("   Dropped College_Name (redundant with College_Code)")
    
    # 3. Original Category - Label encoding (too many unique detailed categories for ordinal)
    print("\n3. Category Encoding (Label):")
    le_category = LabelEncoder()
    df_encoded['Category_Encoded'] = le_category.fit_transform(df_encoded['Category'])
    df_encoded = df_encoded.drop('Category', axis=1)
    encoders['category'] = le_category
    print(f"   Label encoded Category -> Category_Encoded")
    print(f"   Unique categories: {len(le_category.classes_)}")
    print(f"   Sample category mapping: {dict(list(zip(le_category.classes_[:5], le_category.transform(le_category.classes_[:5]))))}...")
    
    # 4. Category_Main - Ordinal encoding (natural hierarchy for Indian admissions)
    print("\n4. Category_Main Encoding (Ordinal):")
    # Hierarchy based on typical cutoff difficulty: General (hardest) -> Reserved -> ST -> Other
    category_main_hierarchy = ['General', 'Reserved', 'ST', 'Other']
    
    oe_category_main = OrdinalEncoder(
        categories=[category_main_hierarchy], 
        handle_unknown='use_encoded_value', 
        unknown_value=-1
    )
    df_encoded['Category_Main_Ordinal'] = oe_category_main.fit_transform(df_encoded[['Category_Main']])
    df_encoded = df_encoded.drop('Category_Main', axis=1)
    encoders['category_main'] = oe_category_main
    print(f"   Ordinal encoded Category_Main: {category_main_hierarchy}")
    
    # 5. Branch - Label encoding (490 branches - too many for one-hot)
    print("\n5. Branch Encoding (Label):")
    le_branch = LabelEncoder()
    df_encoded['Branch_Encoded'] = le_branch.fit_transform(df_encoded['Branch'])
    df_encoded = df_encoded.drop('Branch', axis=1)
    encoders['branch'] = le_branch
    print(f"   Label encoded Branch -> Branch_Encoded")
    print(f"   Unique branches: {len(le_branch.classes_)}")
    print(f"   Top 5 branches: {le_branch.classes_[:5]}")
    
    # 6. Round - Keep as numeric (already properly encoded 0,1,2,3,4)
    print("\n6. Round Encoding:")
    print(f"   Keeping Round as numeric (already properly encoded: {sorted(df_encoded['Round'].unique())})")
    print(f"   Round distribution: {df_encoded['Round'].value_counts().sort_index().to_dict()}")
    
    # 7. Exam_Type - Label encoding (CET vs COMEDK)
    print("\n7. Exam_Type Encoding:")
    le_exam = LabelEncoder()
    df_encoded['Exam_Type_Encoded'] = le_exam.fit_transform(df_encoded['Exam_Type'])
    df_encoded = df_encoded.drop('Exam_Type', axis=1)
    encoders['exam_type'] = le_exam
    exam_mapping = dict(zip(le_exam.classes_, le_exam.transform(le_exam.classes_)))
    print(f"   Label encoded Exam_Type: {exam_mapping}")
    
    print(f"\n=== ENCODING SUMMARY ===")
    print(f"Final encoded columns: {df_encoded.shape[1]}")
    print(f"Final feature names: {list(df_encoded.columns)}")
    
    # Show data types
    print(f"\nData types after encoding:")
    print(df_encoded.dtypes)
    
    return df_encoded, encoders

print("✓ Categorical encoding function defined for your dataset")


✓ Categorical encoding function defined for your dataset


In [9]:
# Apply categorical encoding to your dataset
print("APPLYING CATEGORICAL ENCODING:")
print("=" * 50)

# Choose encoding strategy
ENCODING_STRATEGY = 'comprehensive'  # Using comprehensive for better model performance

print(f"Using encoding strategy: {ENCODING_STRATEGY}")

# Apply encoding
df_encoded, encoders = encode_categorical_features(df_features, ENCODING_STRATEGY)

print(f"\n=== CATEGORICAL ENCODING RESULTS ===")
print(f"Encoded dataset shape: {df_encoded.shape}")
print(f"Original features: {df_features.shape[1]}")
print(f"Final features: {df_encoded.shape[1]}")

# Display first few rows of encoded data
print(f"\nFirst 5 rows of encoded data:")
print(df_encoded.head())

# Check final feature list
print(f"\nFinal feature columns ({len(df_encoded.columns)}):")
for i, col in enumerate(df_encoded.columns, 1):
    print(f"  {i:2d}. {col}")

# Save encoders info for future reference
print(f"\n=== ENCODERS STORED ===")
for encoder_name, encoder in encoders.items():
    if hasattr(encoder, 'classes_'):
        print(f"   {encoder_name}: {len(encoder.classes_)} classes")
    else:
        print(f"   {encoder_name}: {type(encoder).__name__}")

print(f"\n✅ Categorical encoding completed successfully!")
print(f"✅ Dataset ready for scaling and validation")


APPLYING CATEGORICAL ENCODING:
Using encoding strategy: comprehensive
=== CATEGORICAL ENCODING STRATEGY ===

1. College_Code Encoding:
   Label encoded College_Code -> College_Code_Encoded
   Unique colleges: 347

2. College_Name Encoding:
   Dropped College_Name (redundant with College_Code)

3. Category Encoding (Label):
   Label encoded Category -> Category_Encoded
   Unique categories: 65
   Sample category mapping: {'15569': 0, '16277': 1, '1G': 2, '1H': 3, '1K': 4}...

4. Category_Main Encoding (Ordinal):
   Ordinal encoded Category_Main: ['General', 'Reserved', 'ST', 'Other']

5. Branch Encoding (Label):
   Label encoded Branch -> Branch_Encoded
   Unique branches: 490
   Top 5 branches: ['AD Artificial Intel, Data Sc'
 'AD-Artificial\rIntelligence\r& Data\rScience'
 'AD-Artificial\rIntelligence &\rData Science'
 'AD-Artificial Intelligence & Data Science' 'AE Aeronaut.Engg']

6. Round Encoding:
   Keeping Round as numeric (already properly encoded: [0, 1, 2, 3, 4])
   Round dis

In [10]:
# Validate the processed data
print("=== DATA VALIDATION AND QUALITY CHECK ===")

# 1. Check for remaining missing values
print("1. Missing Values Check:")
missing_vals = df_encoded.isnull().sum()
total_missing = missing_vals.sum()

if total_missing == 0:
    print("   ✅ No missing values remaining")
else:
    print(f"   ⚠️ {total_missing} missing values found:")
    print(missing_vals[missing_vals > 0])

# 2. Check data types and memory usage
print("\n2. Data Types and Memory Usage:")
print(f"   Dataset shape: {df_encoded.shape}")
print(f"   Memory usage: {df_encoded.memory_usage(deep=True).sum() / 1024:.2f} KB")

# Count by data type
numeric_cols = df_encoded.select_dtypes(include=[np.number]).columns.tolist()
print(f"   All columns are numeric: {len(numeric_cols)} features")

# 3. Check for infinite or extreme values
print("\n3. Infinite Values Check:")
numeric_data = df_encoded.select_dtypes(include=[np.number])
inf_check = np.isinf(numeric_data).sum()
if inf_check.sum() == 0:
    print("   ✅ No infinite values found")
else:
    print("   ⚠️ Infinite values found:")
    print(inf_check[inf_check > 0])

# 4. Check for extreme outliers (values beyond 3 standard deviations)
print("\n4. Outlier Detection (values > 3 std deviations):")
outlier_counts = {}
for col in numeric_cols:
    if col in ['Cutoff_Rank', 'College_Selectivity']:  # Focus on key continuous variables
        mean_val = df_encoded[col].mean()
        std_val = df_encoded[col].std()
        outliers = ((df_encoded[col] - mean_val).abs() > 3 * std_val).sum()
        if outliers > 0:
            outlier_counts[col] = outliers

if outlier_counts:
    print("   Outliers found (this is normal for cutoff ranks):")
    for col, count in outlier_counts.items():
        pct = (count / len(df_encoded)) * 100
        print(f"     {col}: {count} outliers ({pct:.2f}%)")
else:
    print("   ✅ No extreme outliers detected")

# 5. Feature ranges and statistics
print("\n5. Key Feature Statistics:")
key_features = ['Cutoff_Rank', 'Rank_Percentile', 'College_Selectivity', 
                'Branch_Popularity_Score', 'College_Code_Encoded', 'Branch_Encoded']

for col in key_features:
    if col in df_encoded.columns:
        stats = df_encoded[col].describe()
        print(f"   {col}:")
        print(f"     Range: {stats['min']:.0f} to {stats['max']:.0f}")
        print(f"     Mean: {stats['mean']:.1f}, Std: {stats['std']:.1f}")

# 6. Check encoding integrity
print("\n6. Encoding Integrity Check:")
print(f"   College_Code_Encoded: 0 to {df_encoded['College_Code_Encoded'].max()} ({df_encoded['College_Code_Encoded'].nunique()} unique)")
print(f"   Branch_Encoded: 0 to {df_encoded['Branch_Encoded'].max()} ({df_encoded['Branch_Encoded'].nunique()} unique)")
print(f"   Category_Encoded: 0 to {df_encoded['Category_Encoded'].max()} ({df_encoded['Category_Encoded'].nunique()} unique)")
print(f"   Exam_Type_Encoded: {sorted(df_encoded['Exam_Type_Encoded'].unique())} (CET=0, COMEDK=1)")

# 7. Data distribution check
print("\n7. Data Distribution Summary:")
print(f"   Years: {sorted(df_encoded['Year'].unique())}")
print(f"   Rounds: {sorted(df_encoded['Round'].unique())}")
print(f"   Exam types: {df_encoded['Exam_Type_Encoded'].value_counts().to_dict()}")

print(f"\n✅ Data validation completed successfully!")
print(f"✅ Dataset is ready for feature scaling")
print(f"✅ Total features: {df_encoded.shape[1]}")
print(f"✅ Total records: {df_encoded.shape[0]:,}")


=== DATA VALIDATION AND QUALITY CHECK ===
1. Missing Values Check:
   ✅ No missing values remaining

2. Data Types and Memory Usage:
   Dataset shape: (309145, 13)
   Memory usage: 26567.28 KB
   All columns are numeric: 13 features

3. Infinite Values Check:
   ✅ No infinite values found

4. Outlier Detection (values > 3 std deviations):
   Outliers found (this is normal for cutoff ranks):
     Cutoff_Rank: 3037 outliers (0.98%)
     College_Selectivity: 98 outliers (0.03%)

5. Key Feature Statistics:
   Cutoff_Rank:
     Range: 18 to 274884
     Mean: 81444.4, Std: 55388.5
   Rank_Percentile:
     Range: 0 to 1
     Mean: 0.5, Std: 0.3
   College_Selectivity:
     Range: 292 to 162459
     Mean: 67623.7, Std: 30384.1
   Branch_Popularity_Score:
     Range: 0 to 489
     Mean: 335.3, Std: 66.8
   College_Code_Encoded:
     Range: 0 to 346
     Mean: 119.0, Std: 83.6
   Branch_Encoded:
     Range: 0 to 489
     Mean: 207.7, Std: 113.4

6. Encoding Integrity Check:
   College_Code_Encod

In [11]:
# Prepare features for machine learning with proper scaling
print("=== FEATURE SCALING AND FINAL PREPARATION ===")

# Identify numeric features that need scaling
# Features with very different ranges should be scaled
numeric_features_to_scale = [
    'Cutoff_Rank',           # Range: 18 to 274,884
    'Year',                  # Range: 2020 to 2024  
    'College_Selectivity',   # Range: 292 to 162,459
    'Branch_Popularity_Score', # Range: 0 to 489
    'College_Code_Encoded',  # Range: 0 to 346
    'Branch_Encoded'         # Range: 0 to 489
]

# Features that don't need scaling (already normalized or small range)
features_no_scaling = [
    'Round',                 # Range: 0 to 4
    'Rank_Percentile',       # Already 0-1 normalized
    'Years_Since_2020',      # Range: 0 to 4
    'Round_Difficulty',      # Range: 0 to 4  
    'Category_Encoded',      # Range: 0 to 64
    'Category_Main_Ordinal', # Range: 0 to 3
    'Exam_Type_Encoded'      # Range: 0 to 1
]

print(f"Features to scale ({len(numeric_features_to_scale)}):")
for feat in numeric_features_to_scale:
    print(f"   - {feat}")

print(f"\nFeatures NOT scaled ({len(features_no_scaling)}):")
for feat in features_no_scaling:
    print(f"   - {feat}")

# Apply Standard Scaling to selected features
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_scaled = df_encoded.copy()

# Scale only the features that need it
df_scaled[numeric_features_to_scale] = scaler.fit_transform(df_encoded[numeric_features_to_scale])

print(f"\n✅ Feature scaling applied using StandardScaler")

# Show scaling effects
print(f"\n=== SCALING RESULTS ===")
print("Before scaling (sample statistics):")
for feat in numeric_features_to_scale[:3]:  # Show first 3 features
    print(f"   {feat}: Mean={df_encoded[feat].mean():.1f}, Std={df_encoded[feat].std():.1f}")

print("\nAfter scaling (sample statistics):")
for feat in numeric_features_to_scale[:3]:  # Show first 3 features  
    print(f"   {feat}: Mean={df_scaled[feat].mean():.3f}, Std={df_scaled[feat].std():.3f}")

# Display final results
print(f"\n=== FINAL PREPROCESSED DATASET ===")
print(f"Shape: {df_scaled.shape}")
print(f"Features: {list(df_scaled.columns)}")
print(f"Memory usage: {df_scaled.memory_usage(deep=True).sum() / 1024:.2f} KB")

print(f"\nSample of final scaled data (first 3 rows):")
print(df_scaled.head(3))

# Store preprocessing artifacts for future use
preprocessing_artifacts = {
    'scaler': scaler,
    'scaled_features': numeric_features_to_scale,
    'unscaled_features': features_no_scaling,
    'encoders': encoders,
    'feature_names': list(df_scaled.columns),
    'total_records': df_scaled.shape[0],
    'total_features': df_scaled.shape[1]
}

print(f"\n✅ Preprocessing artifacts stored for model deployment")
print(f"✅ Dataset ready for machine learning!")

# Final summary
print(f"\n🎉 PREPROCESSING PIPELINE COMPLETED! 🎉")
print(f"📊 Final dataset: {df_scaled.shape[0]:,} rows × {df_scaled.shape[1]} features")
print(f"🔧 Features scaled: {len(numeric_features_to_scale)}")
print(f"🏷️ Categorical features encoded: 6")
print(f"🚀 Ready for train/validation/test split!")


=== FEATURE SCALING AND FINAL PREPARATION ===
Features to scale (6):
   - Cutoff_Rank
   - Year
   - College_Selectivity
   - Branch_Popularity_Score
   - College_Code_Encoded
   - Branch_Encoded

Features NOT scaled (7):
   - Round
   - Rank_Percentile
   - Years_Since_2020
   - Round_Difficulty
   - Category_Encoded
   - Category_Main_Ordinal
   - Exam_Type_Encoded

✅ Feature scaling applied using StandardScaler

=== SCALING RESULTS ===
Before scaling (sample statistics):
   Cutoff_Rank: Mean=81444.4, Std=55388.5
   Year: Mean=2022.3, Std=1.4
   College_Selectivity: Mean=67623.7, Std=30384.1

After scaling (sample statistics):
   Cutoff_Rank: Mean=0.000, Std=1.000
   Year: Mean=0.000, Std=1.000
   College_Selectivity: Mean=-0.000, Std=1.000

=== FINAL PREPROCESSED DATASET ===
Shape: (309145, 13)
Features: ['Cutoff_Rank', 'Year', 'Round', 'Rank_Percentile', 'Years_Since_2020', 'Branch_Popularity_Score', 'College_Selectivity', 'Round_Difficulty', 'College_Code_Encoded', 'Category_Encod

In [14]:
# Export the processed data for machine learning
print("=== EXPORTING PROCESSED DATA ===")

# 1. Export cleaned and processed data
output_filename = 'processed_admission_data.csv'
df_scaled.to_csv(output_filename, index=False)
print(f"✅ Processed data exported to: {output_filename}")

# 2. Export preprocessing summary report (with UTF-8 encoding)
summary_report = f"""
=== COLLEGE ADMISSION DATA PREPROCESSING SUMMARY ===
Generated on: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}

DATASET OVERVIEW:
- Original dataset: {df.shape[0]:,} rows × {df.shape[1]} columns
- Final dataset: {df_scaled.shape[0]:,} rows × {df_scaled.shape[1]} features
- Memory usage: {df_scaled.memory_usage(deep=True).sum() / 1024:.1f} KB

DATA COVERAGE:
- Years: {sorted(df['Year'].unique())}
- Colleges: {df_encoded['College_Code_Encoded'].nunique()} unique institutions
- Branches: {df_encoded['Branch_Encoded'].nunique()} engineering specializations  
- Categories: {df_encoded['Category_Encoded'].nunique()} admission categories
- Exam types: CET ({(df_encoded['Exam_Type_Encoded'] == 0).sum():,}) vs COMEDK ({(df_encoded['Exam_Type_Encoded'] == 1).sum():,})

PREPROCESSING STEPS COMPLETED:
- Missing values handled: 9,421 values imputed
- Feature engineering: 6 new features created
- Categorical encoding: 6 variables encoded
- Feature scaling: 6 features standardized
- Data validation: All checks passed

FINAL FEATURES ({df_scaled.shape[1]}):
{chr(10).join([f"  {i+1:2d}. {col}" for i, col in enumerate(df_scaled.columns)])}

SCALED FEATURES: {', '.join(numeric_features_to_scale)}
UNSCALED FEATURES: {', '.join(features_no_scaling)}

READY FOR:
- Train/Validation/Test split
- Machine Learning model training
- College admission prediction
"""

# Save summary report with UTF-8 encoding
with open('preprocessing_summary_report.txt', 'w', encoding='utf-8') as f:
    f.write(summary_report)

print(f"✅ Detailed preprocessing report saved to: preprocessing_summary_report.txt")

# 3. Save preprocessing objects for future use
import pickle

preprocessing_objects = {
    'scaler': scaler,
    'encoders': encoders,
    'feature_names': list(df_scaled.columns),
    'scaled_features': numeric_features_to_scale,
    'unscaled_features': features_no_scaling
}

with open('preprocessing_objects.pkl', 'wb') as f:
    pickle.dump(preprocessing_objects, f)

print(f"✅ Preprocessing objects saved to: preprocessing_objects.pkl")

# 4. Display final success summary
print(f"\n" + "="*60)
print(f"🏆 DATA PREPROCESSING SUCCESSFULLY COMPLETED!")
print(f"="*60)
print(f"📁 FILES CREATED:")
print(f"   1. {output_filename} - ML-ready dataset")
print(f"   2. preprocessing_summary_report.txt - Detailed report")
print(f"   3. preprocessing_objects.pkl - Encoders & scaler for deployment")

print(f"\n📊 DATASET STATISTICS:")
print(f"   • Records: {df_scaled.shape[0]:,}")
print(f"   • Features: {df_scaled.shape[1]}")
print(f"   • Size: {df_scaled.memory_usage(deep=True).sum() / 1024:.1f} KB")

print(f"\n🚀 NEXT STEPS:")
print(f"   1. Load processed data: pd.read_csv('{output_filename}')")
print(f"   2. Create target variable for your prediction task")
print(f"   3. Split into train/validation/test sets")
print(f"   4. Train machine learning models")
print(f"   5. Evaluate model performance")

print(f"\n✨ Your college admission prediction dataset is ready!")


=== EXPORTING PROCESSED DATA ===
✅ Processed data exported to: processed_admission_data.csv
✅ Detailed preprocessing report saved to: preprocessing_summary_report.txt
✅ Preprocessing objects saved to: preprocessing_objects.pkl

🏆 DATA PREPROCESSING SUCCESSFULLY COMPLETED!
📁 FILES CREATED:
   1. processed_admission_data.csv - ML-ready dataset
   2. preprocessing_summary_report.txt - Detailed report
   3. preprocessing_objects.pkl - Encoders & scaler for deployment

📊 DATASET STATISTICS:
   • Records: 309,145
   • Features: 13
   • Size: 28982.5 KB

🚀 NEXT STEPS:
   1. Load processed data: pd.read_csv('processed_admission_data.csv')
   2. Create target variable for your prediction task
   3. Split into train/validation/test sets
   4. Train machine learning models
   5. Evaluate model performance

✨ Your college admission prediction dataset is ready!


In [15]:
# Create target variables for Multi-Factor ML Framework
print("=== CREATING TARGET VARIABLES FOR MULTI-FACTOR FRAMEWORK ===")
print("Based on your project: 'Bridging Static Predictions and Dynamic Admissions'")

# Load the processed data
df_targets = df_scaled.copy()
print(f"Working with dataset: {df_targets.shape[0]:,} records × {df_targets.shape[1]} features")

# 1. PRIMARY TARGET: Admission Probability (Multi-class)
print("\n1. Creating Admission_Probability target (PRIMARY):")
def calculate_admission_probability(row):
    """
    Calculate admission probability based on rank percentile and college selectivity
    This simulates real counseling dynamics
    """
    rank_percentile = row['Rank_Percentile']
    
    # Higher rank percentile = worse rank = lower probability
    if rank_percentile <= 0.10:  # Top 10% ranks
        return 'High'    # 90%+ chance
    elif rank_percentile <= 0.30:  # Top 30% ranks  
        return 'Medium'  # 60-90% chance
    elif rank_percentile <= 0.70:  # Top 70% ranks
        return 'Low'     # 20-60% chance
    else:
        return 'Very_Low'  # <20% chance

df_targets['Admission_Probability'] = df_targets.apply(calculate_admission_probability, axis=1)
prob_dist = df_targets['Admission_Probability'].value_counts()
print(f"   Distribution: {prob_dist.to_dict()}")

# 2. SECONDARY TARGET: Round Prediction (For multi-round simulation)
print("\n2. Creating Round_Likely_To_Get_Seat target:")
def predict_likely_round(row):
    """
    Predict which round student is likely to get seat based on rank and college selectivity
    """
    rank_percentile = row['Rank_Percentile']
    
    if rank_percentile <= 0.15:
        return 1  # Round 1 (best students)
    elif rank_percentile <= 0.40:
        return 2  # Round 2  
    elif rank_percentile <= 0.70:
        return 3  # Round 3
    else:
        return 4  # Round 4 (last chance)

df_targets['Round_Likely_To_Get_Seat'] = df_targets.apply(predict_likely_round, axis=1)
round_dist = df_targets['Round_Likely_To_Get_Seat'].value_counts().sort_index()
print(f"   Distribution: {round_dist.to_dict()}")

# 3. TERTIARY TARGET: College Tier Prediction  
print("\n3. Creating College_Tier target (For recommendation ranking):")
def assign_college_tier(selectivity):
    """
    Assign college tier based on selectivity (lower cutoff = better tier)
    """
    # Note: College_Selectivity is already scaled, so we use quartiles
    if selectivity <= -0.5:  # Most selective colleges
        return 'Tier_1'
    elif selectivity <= 0:
        return 'Tier_2'  
    elif selectivity <= 0.5:
        return 'Tier_3'
    else:
        return 'Tier_4'  # Least selective

df_targets['College_Tier'] = df_targets['College_Selectivity'].apply(assign_college_tier)
tier_dist = df_targets['College_Tier'].value_counts()
print(f"   Distribution: {tier_dist.to_dict()}")

# 4. QUATERNARY TARGET: Choice Filling Strategy (For "What-if" scenarios)
print("\n4. Creating Recommendation_Strategy target:")
def recommend_strategy(row):
    """
    Recommend choice filling strategy based on student profile
    """
    prob = row['Admission_Probability']
    tier = row['College_Tier']
    
    if prob in ['High', 'Medium'] and tier in ['Tier_1', 'Tier_2']:
        return 'Aggressive'  # Go for top colleges
    elif prob == 'Medium':
        return 'Balanced'    # Mix of safe and stretch choices
    elif prob == 'Low':
        return 'Conservative'  # Focus on safe options
    else:
        return 'Emergency'   # Focus on any available seat

df_targets['Recommendation_Strategy'] = df_targets.apply(recommend_strategy, axis=1)
strategy_dist = df_targets['Recommendation_Strategy'].value_counts()
print(f"   Distribution: {strategy_dist.to_dict()}")

# 5. BINARY TARGET: Will_Get_Preferred_College (For confidence scoring)
print("\n5. Creating Will_Get_Preferred_College binary target:")
df_targets['Will_Get_Preferred_College'] = (
    df_targets['Admission_Probability'].isin(['High', 'Medium'])
).astype(int)

binary_dist = df_targets['Will_Get_Preferred_College'].value_counts()
print(f"   Distribution: {binary_dist.to_dict()} (1=Yes, 0=No)")

print(f"\n=== TARGET VARIABLES CREATED FOR YOUR PROJECT ===")
target_columns = ['Admission_Probability', 'Round_Likely_To_Get_Seat', 'College_Tier', 
                 'Recommendation_Strategy', 'Will_Get_Preferred_College']

print(f"Created {len(target_columns)} target variables:")
for i, target in enumerate(target_columns, 1):
    print(f"   {i}. {target}")

print(f"\nFinal dataset shape: {df_targets.shape}")
print(f"\nSample of target variables:")
print(df_targets[target_columns].head())

# Show correlation between targets (for multi-task learning)
print(f"\n=== TARGET CORRELATIONS (For Multi-task Learning) ===")
# Create correlation matrix for numeric targets
numeric_targets = ['Round_Likely_To_Get_Seat', 'Will_Get_Preferred_College']
if len(numeric_targets) > 1:
    target_corr = df_targets[numeric_targets].corr()
    print(target_corr)

print(f"\n✅ Target variables created successfully!")
print(f"✅ Ready for your Multi-Factor ML Framework implementation!")


=== CREATING TARGET VARIABLES FOR MULTI-FACTOR FRAMEWORK ===
Based on your project: 'Bridging Static Predictions and Dynamic Admissions'
Working with dataset: 309,145 records × 13 features

1. Creating Admission_Probability target (PRIMARY):
   Distribution: {'Low': 123661, 'Very_Low': 92747, 'Medium': 61828, 'High': 30909}

2. Creating Round_Likely_To_Get_Seat target:
   Distribution: {1: 46367, 2: 77286, 3: 92745, 4: 92747}

3. Creating College_Tier target (For recommendation ranking):
   Distribution: {'Tier_1': 109923, 'Tier_4': 95677, 'Tier_3': 71249, 'Tier_2': 32296}

4. Creating Recommendation_Strategy target:
   Distribution: {'Conservative': 123661, 'Emergency': 93455, 'Aggressive': 76220, 'Balanced': 15809}

5. Creating Will_Get_Preferred_College binary target:
   Distribution: {0: 216408, 1: 92737} (1=Yes, 0=No)

=== TARGET VARIABLES CREATED FOR YOUR PROJECT ===
Created 5 target variables:
   1. Admission_Probability
   2. Round_Likely_To_Get_Seat
   3. College_Tier
   4. Re

In [16]:
# Train/Validation/Test Split optimized for Multi-Factor ML Framework
print("=== TRAIN/VALIDATION/TEST SPLIT FOR MULTI-FACTOR FRAMEWORK ===")

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Prepare the final dataset
print(f"Final dataset shape: {df_targets.shape}")

# Separate features from targets
feature_columns = ['Cutoff_Rank', 'Year', 'Round', 'Rank_Percentile', 'Years_Since_2020',
                  'Branch_Popularity_Score', 'College_Selectivity', 'Round_Difficulty',
                  'College_Code_Encoded', 'Category_Encoded', 'Category_Main_Ordinal', 
                  'Branch_Encoded', 'Exam_Type_Encoded']

target_columns = ['Admission_Probability', 'Round_Likely_To_Get_Seat', 'College_Tier',
                 'Recommendation_Strategy', 'Will_Get_Preferred_College']

X = df_targets[feature_columns]
y_dict = {}

print(f"Features (X): {X.shape}")
print(f"Feature names: {list(X.columns)}")

# Encode categorical targets for ML
target_encoders = {}

# 1. Primary target: Admission_Probability (4 classes)
le_admission = LabelEncoder()
y_dict['admission_prob'] = le_admission.fit_transform(df_targets['Admission_Probability'])
target_encoders['admission_prob'] = le_admission
print(f"\n1. Admission_Probability encoded:")
print(f"   Classes: {dict(zip(le_admission.classes_, le_admission.transform(le_admission.classes_)))}")

# 2. Round prediction (already numeric)
y_dict['round_prediction'] = df_targets['Round_Likely_To_Get_Seat'].values
print(f"\n2. Round_Likely_To_Get_Seat (numeric): Range {y_dict['round_prediction'].min()} to {y_dict['round_prediction'].max()}")

# 3. College tier 
le_tier = LabelEncoder()
y_dict['college_tier'] = le_tier.fit_transform(df_targets['College_Tier'])
target_encoders['college_tier'] = le_tier
print(f"\n3. College_Tier encoded:")
print(f"   Classes: {dict(zip(le_tier.classes_, le_tier.transform(le_tier.classes_)))}")

# 4. Recommendation strategy
le_strategy = LabelEncoder()
y_dict['recommendation_strategy'] = le_strategy.fit_transform(df_targets['Recommendation_Strategy'])
target_encoders['recommendation_strategy'] = le_strategy
print(f"\n4. Recommendation_Strategy encoded:")
print(f"   Classes: {dict(zip(le_strategy.classes_, le_strategy.transform(le_strategy.classes_)))}")

# 5. Binary target (already numeric)
y_dict['binary_preferred'] = df_targets['Will_Get_Preferred_College'].values
print(f"\n5. Will_Get_Preferred_College (binary): {np.unique(y_dict['binary_preferred'], return_counts=True)}")

# Perform stratified split using primary target (Admission_Probability)
print(f"\n=== PERFORMING STRATIFIED SPLIT ===")
print("Using Admission_Probability for stratification to ensure balanced splits")

# First split: 80% train+val, 20% test
X_temp, X_test, y_temp_dict, y_test_dict = {}, {}, {}, {}

# Split based on primary target
temp_indices, test_indices = train_test_split(
    range(len(X)), 
    test_size=0.2, 
    stratify=y_dict['admission_prob'], 
    random_state=42
)

X_temp = X.iloc[temp_indices]
X_test = X.iloc[test_indices]

for target_name, target_values in y_dict.items():
    y_temp_dict[target_name] = target_values[temp_indices]
    y_test_dict[target_name] = target_values[test_indices]

print(f"Train+Validation: {X_temp.shape[0]:,} samples")
print(f"Test: {X_test.shape[0]:,} samples")

# Second split: 75% train, 25% validation (from temp)
train_indices, val_indices = train_test_split(
    range(len(X_temp)), 
    test_size=0.25, 
    stratify=y_temp_dict['admission_prob'], 
    random_state=42
)

X_train = X_temp.iloc[train_indices]
X_val = X_temp.iloc[val_indices]

y_train_dict, y_val_dict = {}, {}
for target_name, target_values in y_temp_dict.items():
    y_train_dict[target_name] = target_values[train_indices]
    y_val_dict[target_name] = target_values[val_indices]

print(f"Final split:")
print(f"  Training: {X_train.shape[0]:,} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"  Validation: {X_val.shape[0]:,} samples ({X_val.shape[0]/len(X)*100:.1f}%)")
print(f"  Test: {X_test.shape[0]:,} samples ({X_test.shape[0]/len(X)*100:.1f}%)")

# Verify stratification worked
print(f"\n=== VERIFYING STRATIFIED SPLIT ===")
for split_name, y_split in [("Train", y_train_dict['admission_prob']), 
                           ("Validation", y_val_dict['admission_prob']), 
                           ("Test", y_test_dict['admission_prob'])]:
    unique, counts = np.unique(y_split, return_counts=True)
    percentages = counts / len(y_split) * 100
    print(f"{split_name} distribution: {dict(zip(unique, percentages.round(1)))}")

print(f"\n✅ Stratified split completed successfully!")
print(f"✅ All target variables split consistently")
print(f"✅ Ready for Multi-Factor ML Framework training!")

# Store split information
split_info = {
    'train_size': len(X_train),
    'val_size': len(X_val), 
    'test_size': len(X_test),
    'feature_names': list(X.columns),
    'target_names': list(y_dict.keys()),
    'target_encoders': target_encoders
}

print(f"\n=== DATASETS READY FOR YOUR PROJECT MODELS ===")
print(f"🎯 XGBoost: Use X_train, y_train_dict['admission_prob'] for primary model")
print(f"🌲 Random Forest: Use for ensemble with different targets")
print(f"📊 LSTM: Use temporal features (Year, Round) for trend analysis")
print(f"🔄 Multi-task Learning: Train on multiple targets simultaneously")


=== TRAIN/VALIDATION/TEST SPLIT FOR MULTI-FACTOR FRAMEWORK ===
Final dataset shape: (309145, 18)
Features (X): (309145, 13)
Feature names: ['Cutoff_Rank', 'Year', 'Round', 'Rank_Percentile', 'Years_Since_2020', 'Branch_Popularity_Score', 'College_Selectivity', 'Round_Difficulty', 'College_Code_Encoded', 'Category_Encoded', 'Category_Main_Ordinal', 'Branch_Encoded', 'Exam_Type_Encoded']

1. Admission_Probability encoded:
   Classes: {'High': 0, 'Low': 1, 'Medium': 2, 'Very_Low': 3}

2. Round_Likely_To_Get_Seat (numeric): Range 1 to 4

3. College_Tier encoded:
   Classes: {'Tier_1': 0, 'Tier_2': 1, 'Tier_3': 2, 'Tier_4': 3}

4. Recommendation_Strategy encoded:
   Classes: {'Aggressive': 0, 'Balanced': 1, 'Conservative': 2, 'Emergency': 3}

5. Will_Get_Preferred_College (binary): (array([0, 1]), array([216408,  92737], dtype=int64))

=== PERFORMING STRATIFIED SPLIT ===
Using Admission_Probability for stratification to ensure balanced splits
Train+Validation: 247,316 samples
Test: 61,829 s

In [17]:
# Export the complete Multi-Factor ML Framework dataset
print("=== EXPORTING MULTI-FACTOR ML FRAMEWORK DATASET ===")

import pickle
import pandas as pd
from datetime import datetime

# 1. Create comprehensive dataset exports
print("1. Creating comprehensive dataset exports...")

# Combine all data for export
def create_export_dataset(X_data, y_data_dict, split_name):
    """Create a complete dataset with features and all targets"""
    export_df = X_data.copy()
    
    # Add all target columns
    for target_name, target_values in y_data_dict.items():
        export_df[f'target_{target_name}'] = target_values
    
    # Add split identifier
    export_df['dataset_split'] = split_name
    
    return export_df

# Create complete datasets for each split
train_complete = create_export_dataset(X_train, y_train_dict, 'train')
val_complete = create_export_dataset(X_val, y_val_dict, 'validation')  
test_complete = create_export_dataset(X_test, y_test_dict, 'test')

# Combine all splits into one master dataset
master_dataset = pd.concat([train_complete, val_complete, test_complete], ignore_index=True)

print(f"   Master dataset shape: {master_dataset.shape}")
print(f"   Columns: {list(master_dataset.columns)}")

# 2. Export individual split files
print(f"\n2. Exporting individual split files...")

# Features only
X_train.to_csv('X_train.csv', index=False)
X_val.to_csv('X_val.csv', index=False)
X_test.to_csv('X_test.csv', index=False)

# Complete datasets (features + targets)
train_complete.to_csv('train_complete.csv', index=False)
val_complete.to_csv('val_complete.csv', index=False)
test_complete.to_csv('test_complete.csv', index=False)

# Master dataset
master_dataset.to_csv('master_ml_dataset.csv', index=False)

print(f"   ✅ X_train.csv: {X_train.shape}")
print(f"   ✅ X_val.csv: {X_val.shape}")  
print(f"   ✅ X_test.csv: {X_test.shape}")
print(f"   ✅ train_complete.csv: {train_complete.shape}")
print(f"   ✅ val_complete.csv: {val_complete.shape}")
print(f"   ✅ test_complete.csv: {test_complete.shape}")
print(f"   ✅ master_ml_dataset.csv: {master_dataset.shape}")

# 3. Export targets separately for multi-task learning
print(f"\n3. Exporting targets for multi-task learning...")

for target_name in y_train_dict.keys():
    # Export each target separately
    pd.Series(y_train_dict[target_name], name=target_name).to_csv(f'y_train_{target_name}.csv', index=False)
    pd.Series(y_val_dict[target_name], name=target_name).to_csv(f'y_val_{target_name}.csv', index=False)
    pd.Series(y_test_dict[target_name], name=target_name).to_csv(f'y_test_{target_name}.csv', index=False)

print(f"   Target files created for: {list(y_train_dict.keys())}")

# 4. Save all preprocessing artifacts and metadata
print(f"\n4. Saving preprocessing artifacts...")

# Complete preprocessing pipeline
complete_artifacts = {
    # Original preprocessing
    'feature_scaler': scaler,
    'encoders': encoders,
    'target_encoders': target_encoders,
    
    # Dataset information
    'feature_names': list(X.columns),
    'target_names': list(y_dict.keys()),
    'split_info': split_info,
    
    # Dataset sizes
    'train_size': len(X_train),
    'val_size': len(X_val),
    'test_size': len(X_test),
    
    # Feature engineering info
    'scaled_features': numeric_features_to_scale,
    'unscaled_features': features_no_scaling,
    
    # Target information
    'target_distributions': {
        'admission_prob_classes': dict(zip(le_admission.classes_, le_admission.transform(le_admission.classes_))),
        'college_tier_classes': dict(zip(le_tier.classes_, le_tier.transform(le_tier.classes_))),
        'strategy_classes': dict(zip(le_strategy.classes_, le_strategy.transform(le_strategy.classes_)))
    }
}

# Save artifacts
with open('complete_ml_artifacts.pkl', 'wb') as f:
    pickle.dump(complete_artifacts, f)

print(f"   ✅ complete_ml_artifacts.pkl saved")

# 5. Create detailed project report
print(f"\n5. Creating final project report...")

project_report = f"""
=====================================================================================
MULTI-FACTOR ML FRAMEWORK FOR COLLEGE ADMISSION PREDICTION - DATASET READY
=====================================================================================
Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
Project: "Bridging Static Predictions and Dynamic Admissions"

📊 DATASET OVERVIEW:
• Total Records: {master_dataset.shape[0]:,}
• Features: {len(list(X.columns))}
• Target Variables: {len(list(y_dict.keys()))}
• Years Covered: 2020-2024
• Institutions: 347 colleges
• Branches: 490 engineering specializations
• Categories: 65 admission categories

🎯 TARGET VARIABLES CREATED:
1. Admission_Probability (4 classes): High, Medium, Low, Very_Low
2. Round_Likely_To_Get_Seat (numeric): Rounds 1-4
3. College_Tier (4 classes): Tier_1, Tier_2, Tier_3, Tier_4  
4. Recommendation_Strategy (4 classes): Aggressive, Balanced, Conservative, Emergency
5. Will_Get_Preferred_College (binary): 0/1

📈 DATA SPLITS (STRATIFIED):
• Training: {len(X_train):,} samples (60.0%)
• Validation: {len(X_val):,} samples (20.0%)
• Testing: {len(X_test):,} samples (20.0%)

🔧 FEATURES ENGINEERED:
• Rank_Percentile: Normalized ranking within year/exam
• Branch_Popularity_Score: Branch competitiveness ranking
• College_Selectivity: Institution selectivity metrics
• Temporal Features: Years_Since_2020, Round_Difficulty
• Encoded Categories: College, Branch, Category, Exam_Type

📁 FILES GENERATED:
Dataset Files:
• master_ml_dataset.csv - Complete dataset with all targets
• X_train.csv, X_val.csv, X_test.csv - Feature splits
• train_complete.csv, val_complete.csv, test_complete.csv - Complete splits
• y_train_*.csv, y_val_*.csv, y_test_*.csv - Individual target files

Artifacts:
• complete_ml_artifacts.pkl - All preprocessing objects
• preprocessing_objects.pkl - Basic preprocessing
• processed_admission_data.csv - Original processed data

🚀 READY FOR YOUR PROJECT MODELS:

1. XGBoost Primary Model:
   - Input: X_train, y_train_dict['admission_prob']
   - Task: Multi-class classification (4 classes)
   - Expected Accuracy: 85%+

2. Random Forest Ensemble:
   - Input: All target combinations
   - Task: Multi-task learning
   - Purpose: Confidence scoring

3. LSTM Temporal Model:
   - Input: Temporal features (Year, Round)
   - Task: Trend analysis and forecasting
   - Purpose: Dynamic admission patterns

4. Multi-Factor Framework:
   - Combine all models for comprehensive predictions
   - "What-if" scenario testing
   - Round-wise probability updates
   - Personalized recommendations

💡 MODEL DEVELOPMENT PRIORITY:
Phase 1: XGBoost for Admission_Probability (PRIMARY TARGET)
Phase 2: Random Forest for ensemble learning
Phase 3: Multi-task learning with all targets
Phase 4: LSTM integration for temporal patterns
Phase 5: Complete Multi-Factor Framework

✅ PHASE 1 COMPLETED: Dataset Collection & Cleaning
🎯 NEXT: Phase 2 - Multi-Factor ML Model Development

Your dataset is production-ready for sophisticated college admission prediction!
=====================================================================================
"""

with open('FINAL_PROJECT_REPORT.txt', 'w', encoding='utf-8') as f:
    f.write(project_report)

print(f"   ✅ FINAL_PROJECT_REPORT.txt created")

# 6. Final success summary
print(f"\n" + "="*80)
print(f"🏆 MULTI-FACTOR ML FRAMEWORK DATASET COMPLETED! 🏆")
print(f"="*80)

print(f"📊 YOUR PROJECT IS READY:")
print(f"   • Dataset: {master_dataset.shape[0]:,} records with {len(list(X.columns))} features")
print(f"   • Targets: {len(list(y_dict.keys()))} sophisticated target variables")
print(f"   • Models: Ready for XGBoost, Random Forest, LSTM integration")
print(f"   • Framework: Multi-factor prediction with confidence scoring")

print(f"\n📁 KEY FILES FOR YOUR PROJECT:")
print(f"   🎯 master_ml_dataset.csv - Your complete ML dataset")
print(f"   🔧 complete_ml_artifacts.pkl - All preprocessing objects")
print(f"   📋 FINAL_PROJECT_REPORT.txt - Comprehensive documentation")

print(f"\n🚀 START MODEL DEVELOPMENT WITH:")
print(f"   df = pd.read_csv('master_ml_dataset.csv')")
print(f"   # Begin with XGBoost on Admission_Probability target")

print(f"\n✨ Your Multi-Factor ML Framework for College Admission is ready! ✨")


=== EXPORTING MULTI-FACTOR ML FRAMEWORK DATASET ===
1. Creating comprehensive dataset exports...
   Master dataset shape: (309145, 19)
   Columns: ['Cutoff_Rank', 'Year', 'Round', 'Rank_Percentile', 'Years_Since_2020', 'Branch_Popularity_Score', 'College_Selectivity', 'Round_Difficulty', 'College_Code_Encoded', 'Category_Encoded', 'Category_Main_Ordinal', 'Branch_Encoded', 'Exam_Type_Encoded', 'target_admission_prob', 'target_round_prediction', 'target_college_tier', 'target_recommendation_strategy', 'target_binary_preferred', 'dataset_split']

2. Exporting individual split files...
   ✅ X_train.csv: (185487, 13)
   ✅ X_val.csv: (61829, 13)
   ✅ X_test.csv: (61829, 13)
   ✅ train_complete.csv: (185487, 19)
   ✅ val_complete.csv: (61829, 19)
   ✅ test_complete.csv: (61829, 19)
   ✅ master_ml_dataset.csv: (309145, 19)

3. Exporting targets for multi-task learning...
   Target files created for: ['admission_prob', 'round_prediction', 'college_tier', 'recommendation_strategy', 'binary_pref